In [ ]:
import os
import arcpy
import pandas as pd
import numpy as np
import csv
from arcgis.features import GeoAccessor, GeoSeriesAccessor

arcpy.env.overwriteOutput = True

<h4>Creación de DataFrame para la unificación de la tabla -Capa-</h4>

In [ ]:
df_unificacion_tabla_capa = pd.DataFrame(columns=['nombre_capa_final', 'codigo_tematica','CODDANE'])
print("---------------------------------------- Data frame creado ----------------------------------------")

<h4>Generación de DataFrame Global Total con la unificación de las tres columnas principales</h4>
<ul>
  <li>nombre_capa_final</li>
  <li>codigo_tematica</li>
  <li>coddane</li>
</ul>

In [ ]:
ruta_capas_w_municipio = r"C:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\Local_BD_PGAR.gdb"

arcpy.env.workspace = ruta_capas_w_municipio

dataset_intersect = 'Interseccion_Municipio'

for capas_w_mpio in arcpy.ListFeatureClasses(feature_dataset = dataset_intersect):
    ruta_capas_w_mpio = os.path.join(ruta_capas_w_municipio,capas_w_mpio)
    capas_w_mpio = pd.DataFrame.spatial.from_featureclass(ruta_capas_w_mpio)
    capas_w_mpio_columnas_reducidas = capas_w_mpio.loc[:, ['nombre_capa_final','codigo_tematica','CODDANE']]
    df_unificacion_tabla_capa = df_unificacion_tabla_capa.append(capas_w_mpio_columnas_reducidas)

<h4>Generación del campo -Código Capa-</h4>

In [ ]:
# Agrupamiento, lo que en SQL es Group by
df_tabla_capa = df_unificacion_tabla_capa.groupby(['nombre_capa_final','codigo_tematica']).agg({'nombre_capa_final':'count', 'codigo_tematica':'count'})

# Generación del campo incremental de acuerdo al número de capas
codigo_capa = []

# Se suma uno por el header dentro del data frame
for numero_capas in range(1, df_tabla_capa.shape[0]+1):
    codigo_capa.append(numero_capas)
 
# Se utiliza el método .loc para incorporar la columna, que en este caso fue la lista que se creo previamente   
df_tabla_capa.loc[:,'codigo_capa'] = codigo_capa

# Se cambia el nombre de las columnas asociadas a los conteos (funcion -rename-)
df_tabla_capa.rename({'codigo_tematica':'conteo_tematica', 'nombre_capa_final': 'conteo_nombre_capa_final'}, axis = 1, inplace = True)

# Se resetan los índices
df_tabla_capa_completa = df_tabla_capa.reset_index()

# Se estandariza la tabla final
df_capa = df_tabla_capa_completa[['codigo_capa','nombre_capa_final','codigo_tematica']]

<h4>Generación de Diccionario para parametrizar el -codigo_capa-</h4>

In [ ]:
# Parametrización del diccionario de las capas
df_capa_parametrizacion_previa_diccionario = df_capa[['codigo_capa','nombre_capa_final']]

# Selección de nuevo índice
df_capa_parametrizacion_previa_diccionario = df_capa_parametrizacion_previa_diccionario.set_index('codigo_capa')

# Migración de Data Frame a Diccionario -función to_dict()-
diccionario_capas = df_capa_parametrizacion_previa_diccionario.to_dict()

# Modelado del diccionario tradicional
diccionario_capas_definitivo = diccionario_capas['nombre_capa_final']

<h4>Actualización de cada capa intersecada con Municipio con su respectivo -codigo_capa-</h4>

In [ ]:
nombre_campo_id_capa = 'codigo_capa'

for capas_w_mpio in arcpy.ListFeatureClasses(feature_dataset = dataset_intersect):
    if arcpy.ListFields(capas_w_mpio,nombre_campo_id_capa):
        print("El campo existe en la capa: {0}".format(capas_w_mpio))
    else:
        arcpy.management.AddField(capas_w_mpio, nombre_campo_id_capa, 'DOUBLE', field_alias = 'Codigo Capa')
        for llave, valor in diccionario_capas_definitivo.items():
            if capas_w_mpio == valor + '_Intersect_Mpio':
                arcpy.management.CalculateField(capas_w_mpio, nombre_campo_id_capa, llave, 'PYTHON3')
                print("Se actualiza el -Codigo de la Capa-, en el feature class: {0}".format(capas_w_mpio))

<h4>Incorporación de columnas -codigo_fuente- y -fecha_ultima_actualizacion- a la tabla -tbl_capa-</h4>

In [ ]:
# --- Carga Feature Table -tbl_fuentes- --
nombre_tabla_fuente = 'tbl_fuentes'
ruta_tabla_fuente = os.path.join(arcpy.env.workspace,nombre_tabla_fuente)
df_fuente = pd.DataFrame.spatial.from_table(ruta_tabla_fuente)

# --- Carga .csv con METADATOS --
df_otros_metadatos_insumo = pd.read_csv("otros_metadatos.csv", sep = ";")
df_otros_metadatos_insumo_filtro = df_otros_metadatos_insumo[df_otros_metadatos_insumo['nombre_capa'].str.slice(0, 1) == '_']

# Estrucuracion
df_fuente_insumo_estructurada = df_otros_metadatos_insumo_filtro.loc[:,['nombre_capa','ultima_actualizacion_reportada','fuente','descripcion_capa']]

# Primer JOIN: Busca acceder a la fecha de actualización y al nombre de la fuente
join_capa_metadato = pd.merge(df_capa, df_fuente_insumo_estructurada, how = 'left', left_on = 'nombre_capa_final', right_on = 'nombre_capa')

# Parametrización del DF
join_capa_metadato = join_capa_metadato.loc[:,['codigo_capa','nombre_capa_final','ultima_actualizacion_reportada','fuente','descripcion_capa','codigo_tematica']]

# Segundo JOIN: Busca acceder al código de la fuente
join_capa_fuente = pd.merge(join_capa_metadato, df_fuente, how = 'left', left_on = 'fuente', right_on = 'nombre_fuente')

# Parametrización del DF
join_capa_fuente = join_capa_fuente.loc[:, ['codigo_capa','nombre_capa_final','ultima_actualizacion_reportada','codigo_fuente','descripcion_capa','codigo_tematica']]

# Se cambian nombres de columnas
join_capa_fuente.rename({'nombre_capa_final': 'nombre_capa', 'ultima_actualizacion_reportada':'fecha_actualizacion_capa'}, axis = 1, inplace = True)

# Reemplazar por 0 todo NaN. Se usa la función fillna
join_capa_fuente['codigo_fuente'].fillna(0, inplace=True)

# Cambio código a entero
join_capa_fuente['codigo_fuente'] = join_capa_fuente['codigo_fuente'].astype('double')

<h4>Exportar capa -tbl_capa- a FileGDB</h4>

In [ ]:
nombre_tabla_capa = 'tbl_capa'
ruta_salida_tabla_capa = os.path.join(arcpy.env.workspace, nombre_tabla_capa)
join_capa_fuente.spatial.to_table(location = ruta_salida_tabla_capa)